## Creates the doc2vec vector embeddings for a specific configuration

In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import io
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random
import gzip

from multiprocessing import Process, Queue

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *
from thesis.utils.file import *

## Global variables used throughout the script

In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234

In [4]:
MIN_WORD_COUNT = 100
NUM_CORES = 16

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [7]:
root_location = "/mnt/virtual-machines/data/"
exports_location = root_location + "exported_data/"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/extended_pv_abs_desc_claims_full_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

## Load general data required for classification

In [8]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 17.7 s, sys: 1.08 s, total: 18.8 s
Wall time: 18.8 s


In [9]:
len(training_docs_list)

1286325

In [10]:
len(validation_docs_list)

321473

In [11]:
len(test_docs_list)

401877

# Utility functions for data loading

In [12]:
VALIDATION_MINI_BATCH_SIZE = 10000
def get_extended_docs_with_inference_data_only(doc2vec_model, file_to_write, preprocessed_files_prefix, level, model_name):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)

    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_documents_reps = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
    else:
        inference_documents_reps = {}
        info("===== Getting vectors with inference")

        # Multi-threaded inference
#         inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        inference_docs_iterator = BatchWrapper(preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)
        generator_func = inference_docs_iterator.__iter__()
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        batches_run = 1
        pool = ThreadPool(NUM_CORES)
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(batches_run * mini_batch_size))
            batches_run += 1
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break
                
        pool.close()
        pool.terminate()

        pickle.dump(inference_documents_reps,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_documents_reps

In [13]:
class ExtendedPVDocumentBatchGenerator(Process):
    def __init__(self, filename_prefix, queue, batch_size=10000, start_file=0, offset=10000):
        super(ExtendedPVDocumentBatchGenerator, self).__init__()
        self.queue = queue
        self.offset = offset
        self.filename_prefix = filename_prefix
        self.files_loaded = start_file - offset

    def run(self):
        cur_file = None
        while True:
            try:
                if cur_file is None:
                    info("Loading new file for index: {}".format(str(self.files_loaded + self.offset)))
                    cur_file = io.BufferedReader(gzip.open(self.filename_prefix + str(self.files_loaded + self.offset) + '.gz'))
#                     cur_file = open(self.filename_prefix + str(self.files_loaded + self.offset))
                    self.files_loaded += self.offset
                for line in cur_file:
                    self.queue.put(line)
                cur_file.close()
                cur_file = None
            except IOError:
                self.queue.put(False, block=True, timeout=None)
                info("All files are loaded - last file: {}".format(str(self.files_loaded + self.offset)))
                return


class BatchWrapper(object):
    def __init__(self, training_preprocessed_files_prefix, buffer_size=10000, batch_size=10000, level=1, level_type=None):
        assert batch_size <= 10000 or batch_size is None
        self.level = level
        self.level_type = level_type[0] if level_type is not None else None
        self.batch_size = batch_size
        self.q = Queue(maxsize=buffer_size)
        self.p = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, queue=self.q,
                                                  batch_size=batch_size, start_file=0, offset=10000)
        self.p.start()
        self.cur_data = []

    def is_correct_type(self, doc_id):
        parts = doc_id.split("_")
        len_parts = len(parts)
        if len_parts == self.level:
            if len_parts == 1:
                return True
            if len_parts == self.level and (parts[1][0] == self.level_type or self.level_type is None):
                return True
        return False

    def return_sentences(self, line):
        line_array = tuple(line.split(" "))
        doc_id = line_array[0]
        if not self.is_correct_type(doc_id):
            return False
        line_array = line_array[1:]
        len_line_array = len(line_array)
        # divide the document to batches according to the batch size
        sentences = []
        
        if self.batch_size is None:
            # dont use LabeledSentence for validation iterator
            sentences.append((doc_id, line_array))
        else:
            curr_batch_iter = 0
            while curr_batch_iter < len_line_array:
                sentences.append(LabeledSentence(words=line_array[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id]))
                curr_batch_iter += self.batch_size
        return tuple(sentences)

    def __iter__(self):
        while True:
            item = self.q.get(block=True)
            if item is False:
                self.p.terminate()
                raise StopIteration()
            else:
                sentences = self.return_sentences(item)
                if not sentences:
                    None
                else:
                    for sentence in sentences:
                        yield sentence


# Doc2vec and SVM Parameters

In [14]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

## Create the Doc2vec model and create/load the vocab

In [15]:
models = [
    (2, 'description')
]
level, model_name = models[0]

In [18]:
info("creating/loading vocabulary for " + str(level) + ' ' + model_name + ' in ')
doc2vec_model_save_location = os.path.join(root_location,
                                           "parameter_search_doc2vec_models_recalc_" + str(level) + '_' + model_name,
                                           "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}_model_{}'.format(DOC2VEC_SIZE,
                                                                DOC2VEC_WINDOW,
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE),
                                                                str(level) + '_' + model_name
                                                                )
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
info("FILE " + os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
doc2vec_model = Doc2Vec(size=DOC2VEC_SIZE, window=DOC2VEC_WINDOW, min_count=MIN_WORD_COUNT,
                max_vocab_size= DOC2VEC_MAX_VOCAB_SIZE,
                sample=DOC2VEC_SAMPLE, seed=DOC2VEC_SEED, workers=NUM_CORES,
                # doc2vec algorithm dm=1 => PV-DM, dm=2 => PV-DBOW, PV-DM dictates CBOW for words
                dm=DOC2VEC_TYPE,
                # hs=0 => negative sampling, hs=1 => hierarchical softmax
                hs=DOC2VEC_HIERARCHICAL_SAMPLE, negative=DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                dm_concat=DOC2VEC_CONCAT,
                # would train words with skip-gram on top of cbow, we don't need that for now
                dbow_words=DOC2VEC_TRAIN_WORDS,
                iter=DOC2VEC_EPOCHS)

GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model


2017-04-19 02:01:54,634 : INFO : creating/loading vocabulary for 2 description in 
2017-04-19 02:01:54,636 : INFO : FILE /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_description/full/vocab_model/model


In [19]:
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX)):
    
    info("Creating vocab model")
    training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
    doc2vec_model.build_vocab(sentences=training_docs_iterator, progress_per=REPORT_VOCAB_PROGRESS)
    doc2vec_model.save(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
else:
    info("Loading vocab model")
    doc2vec_model_vocab_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, VOCAB_MODEL, MODEL_PREFIX))
    doc2vec_model.reset_from(doc2vec_model_vocab_model)

2017-04-19 02:01:56,803 : INFO : Creating vocab model
2017-04-19 02:01:56,846 : INFO : collecting all words and their counts
2017-04-19 02:01:56,849 : INFO : Loading new file for index: 0
2017-04-19 02:01:56,856 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-19 02:02:26,877 : INFO : Loading new file for index: 10000
2017-04-19 02:02:57,273 : INFO : Loading new file for index: 20000
2017-04-19 02:03:29,916 : INFO : Loading new file for index: 30000
2017-04-19 02:04:02,779 : INFO : Loading new file for index: 40000
2017-04-19 02:04:35,457 : INFO : Loading new file for index: 50000
2017-04-19 02:05:08,435 : INFO : Loading new file for index: 60000
2017-04-19 02:05:40,060 : INFO : Loading new file for index: 70000
2017-04-19 02:05:48,746 : INFO : PROGRESS: at example #100000, processed 626223826 words (2700540/s), 3125696 word types, 72315 tags
2017-04-19 02:06:13,408 : INFO : Loading new file for index: 80000
2017-04-19 02:06:46,458 : INFO : Loadin

2017-04-19 02:52:10,590 : INFO : Loading new file for index: 940000
2017-04-19 02:52:25,230 : INFO : PROGRESS: at example #1300000, processed 8134258763 words (2440725/s), 20168634 word types, 944012 tags
2017-04-19 02:52:44,923 : INFO : Loading new file for index: 950000
2017-04-19 02:53:18,893 : INFO : Loading new file for index: 960000
2017-04-19 02:53:53,487 : INFO : Loading new file for index: 970000
2017-04-19 02:54:27,993 : INFO : Loading new file for index: 980000
2017-04-19 02:55:02,293 : INFO : Loading new file for index: 990000
2017-04-19 02:55:36,997 : INFO : Loading new file for index: 1000000
2017-04-19 02:56:11,641 : INFO : Loading new file for index: 1010000
2017-04-19 02:56:35,940 : INFO : PROGRESS: at example #1400000, processed 8759559554 words (2494142/s), 21243867 word types, 1016608 tags
2017-04-19 02:56:46,434 : INFO : Loading new file for index: 1020000
2017-04-19 02:57:21,064 : INFO : Loading new file for index: 1030000
2017-04-19 02:57:57,479 : INFO : Loading 

## Actual Training, validation and Metrics Loop

In [20]:
doc2vec_model.min_alpha = 0.025
DOC2VEC_ALPHA_DECREASE = 0.001

In [21]:
doc2vec_model.workers = NUM_CORES

In [22]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 1
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
        training_docs_iterator = BatchWrapper(training_preprocessed_files_prefix, batch_size=10000, level=level,
                                          level_type=model_name)
        %time doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
    # only do the inference for higher epochs, as inference usually takes as much time as the actual training
    if epoch == 3 or epoch == 5:
        # Validation Embeddings
        info('Getting Validation Embeddings')
        Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-19 03:13:21,923 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_1 *******************
2017-04-19 03:13:22,070 : INFO : training model with 16 workers on 424864 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=10 window=2
2017-04-19 03:13:22,072 : INFO : expecting 1770337 sentences, matching count from corpus used for vocabulary survey
2017-04-19 03:13:22,071 : INFO : Loading new file for index: 0
2017-04-19 03:13:23,097 : INFO : PROGRESS: at 0.01% examples, 495978 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:13:43,112 : INFO : PROGRESS: at 0.18% examples, 656700 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:14:03,131 : INFO : PROGRESS: at 0.37% examples, 672721 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:14:23,139 : INFO : PROGRESS: at 0.56% examples, 679811 words/s, in_qsize 0, out_qsize 3
2017-04-19 03:14:43,144 : INFO : PROGRESS: at 0.74% 

2017-04-19 03:35:23,713 : INFO : PROGRESS: at 12.46% examples, 710113 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:35:23,853 : INFO : Loading new file for index: 160000
2017-04-19 03:35:43,719 : INFO : PROGRESS: at 12.65% examples, 710368 words/s, in_qsize 0, out_qsize 2
2017-04-19 03:36:03,726 : INFO : PROGRESS: at 12.85% examples, 710538 words/s, in_qsize 0, out_qsize 2
2017-04-19 03:36:23,732 : INFO : PROGRESS: at 13.04% examples, 710651 words/s, in_qsize 0, out_qsize 2
2017-04-19 03:36:43,732 : INFO : PROGRESS: at 13.23% examples, 710752 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:36:44,286 : INFO : Loading new file for index: 170000
2017-04-19 03:37:03,742 : INFO : PROGRESS: at 13.42% examples, 710872 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:37:23,746 : INFO : PROGRESS: at 13.61% examples, 710969 words/s, in_qsize 1, out_qsize 0
2017-04-19 03:37:43,747 : INFO : PROGRESS: at 13.80% examples, 711074 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:38:03,748 : INFO : PROGRES

2017-04-19 03:58:22,028 : INFO : Loading new file for index: 330000
2017-04-19 03:58:24,179 : INFO : PROGRESS: at 25.71% examples, 716291 words/s, in_qsize 0, out_qsize 1
2017-04-19 03:58:44,187 : INFO : PROGRESS: at 25.91% examples, 716358 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:59:04,190 : INFO : PROGRESS: at 26.10% examples, 716414 words/s, in_qsize 0, out_qsize 1
2017-04-19 03:59:24,193 : INFO : PROGRESS: at 26.30% examples, 716514 words/s, in_qsize 0, out_qsize 0
2017-04-19 03:59:41,248 : INFO : Loading new file for index: 340000
2017-04-19 03:59:44,195 : INFO : PROGRESS: at 26.49% examples, 716605 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:00:04,196 : INFO : PROGRESS: at 26.69% examples, 716659 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:00:24,196 : INFO : PROGRESS: at 26.88% examples, 716714 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:00:44,216 : INFO : PROGRESS: at 27.07% examples, 716755 words/s, in_qsize 0, out_qsize 2
2017-04-19 04:01:00,747 : INFO : Loading

2017-04-19 04:21:04,596 : INFO : PROGRESS: at 38.92% examples, 720212 words/s, in_qsize 0, out_qsize 2
2017-04-19 04:21:24,599 : INFO : PROGRESS: at 39.12% examples, 720238 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:21:44,599 : INFO : PROGRESS: at 39.31% examples, 720278 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:22:04,603 : INFO : PROGRESS: at 39.51% examples, 720363 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:22:20,678 : INFO : Loading new file for index: 510000
2017-04-19 04:22:24,609 : INFO : PROGRESS: at 39.70% examples, 720425 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:22:44,619 : INFO : PROGRESS: at 39.90% examples, 720545 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:23:04,621 : INFO : PROGRESS: at 40.10% examples, 720644 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:23:24,623 : INFO : PROGRESS: at 40.30% examples, 720715 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:23:39,378 : INFO : Loading new file for index: 520000
2017-04-19 04:23:44,622 : INFO : PROGRES

2017-04-19 04:44:05,194 : INFO : PROGRESS: at 52.42% examples, 723489 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:44:25,198 : INFO : PROGRESS: at 52.61% examples, 723519 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:44:45,210 : INFO : PROGRESS: at 52.81% examples, 723562 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:44:47,687 : INFO : Loading new file for index: 680000
2017-04-19 04:45:05,214 : INFO : PROGRESS: at 53.00% examples, 723593 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:45:25,226 : INFO : PROGRESS: at 53.20% examples, 723631 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:45:45,231 : INFO : PROGRESS: at 53.39% examples, 723704 words/s, in_qsize 0, out_qsize 0
2017-04-19 04:46:05,233 : INFO : PROGRESS: at 53.59% examples, 723731 words/s, in_qsize 0, out_qsize 1
2017-04-19 04:46:08,328 : INFO : Loading new file for index: 690000
2017-04-19 04:46:25,233 : INFO : PROGRESS: at 53.78% examples, 723768 words/s, in_qsize 0, out_qsize 2
2017-04-19 04:46:45,238 : INFO : PROGRES

...

2017-04-20 03:58:40,443 : INFO : PROGRESS: at 86.83% examples, 831791 words/s, in_qsize 0, out_qsize 0
2017-04-20 03:59:00,444 : INFO : PROGRESS: at 87.05% examples, 831843 words/s, in_qsize 0, out_qsize 0
2017-04-20 03:59:04,028 : INFO : Loading new file for index: 1120000
2017-04-20 03:59:20,453 : INFO : PROGRESS: at 87.28% examples, 831900 words/s, in_qsize 2, out_qsize 0
2017-04-20 03:59:40,453 : INFO : PROGRESS: at 87.51% examples, 831933 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:00:00,483 : INFO : PROGRESS: at 87.73% examples, 831988 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:00:13,689 : INFO : Loading new file for index: 1130000
2017-04-20 04:00:20,484 : INFO : PROGRESS: at 87.96% examples, 832041 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:00:40,486 : INFO : PROGRESS: at 88.18% examples, 832077 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:01:00,490 : INFO : PROGRESS: at 88.41% examples, 832123 words/s, in_qsize 0, out_qsize 0
2017-04-20 04:01:20,501 : INFO : PROGR

2017-04-20 04:17:57,202 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-04-20 04:17:57,203 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-04-20 04:17:57,211 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-04-20 04:17:57,219 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-20 04:17:57,223 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-04-20 04:17:57,224 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-04-20 04:17:57,225 : INFO : training on 11070832244 raw words (7509452256 effective words) took 9000.2s, 834362 effective words/s
2017-04-20 04:17:57,226 : INFO : saving Doc2Vec object under /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_8/model, separately None
2017-04-20 04:17:57,227 : INFO : stor

CPU times: user 17h 32min 25s, sys: 10min 12s, total: 17h 42min 38s
Wall time: 2h 30min


2017-04-20 04:17:57,952 : INFO : not storing attribute syn0norm
2017-04-20 04:17:57,953 : INFO : storing np array 'syn0' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_8/model.wv.syn0.npy
2017-04-20 04:17:58,165 : INFO : storing np array 'syn1neg' to /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_8/model.syn1neg.npy
2017-04-20 04:17:58,349 : INFO : not storing attribute cum_table
2017-04-20 04:18:11,474 : INFO : saved /mnt/virtual-machines/data/parameter_search_doc2vec_models_recalc_2_description/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_description/epoch_8/model


CPU times: user 7d 6h 40min 58s, sys: 1h 53min 55s, total: 7d 8h 34min 54s
Wall time: 1d 1h 4min 49s


## Inference Only (if needed)

In [42]:
NUM_CORES = 32

In [41]:
epoch = 1
GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)

if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
    doc2vec_model.workers = NUM_CORES
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model

2017-04-18 18:58:38,660 : INFO : loading Doc2Vec object from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_1/model
2017-04-18 18:58:45,944 : INFO : loading docvecs recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_1/model.docvecs.* with mmap=None
2017-04-18 18:58:45,945 : INFO : loading doctag_syn0 from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_1/model.docvecs.doctag_syn0.npy with mmap=None
2017-04-18 18:58:51,024 : INFO : loading wv recursively from /mnt/virtual-machines/data/parameter_search_doc2vec_models_2_claims/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_

In [43]:
GLOBAL_VARS.MODEL_NAME

'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None_model_2_claims/epoch_1'

In [ ]:
Xv = get_extended_docs_with_inference_data_only(doc2vec_model, VALIDATION_DICT, 
                                         validation_preprocessed_files_prefix, level, model_name)

2017-04-18 18:59:26,842 : INFO : ===== Getting vectors with inference
2017-04-18 18:59:27,021 : INFO : Loading new file for index: 0
2017-04-18 18:59:55,443 : INFO : Loading new file for index: 10000
2017-04-18 19:00:44,049 : INFO : Finished: 10000 tags
2017-04-18 19:01:10,981 : INFO : Loading new file for index: 20000
2017-04-18 19:01:52,567 : INFO : Finished: 20000 tags
2017-04-18 19:02:18,405 : INFO : Loading new file for index: 30000
2017-04-18 19:02:58,946 : INFO : Finished: 30000 tags
2017-04-18 19:03:25,179 : INFO : Loading new file for index: 40000
2017-04-18 19:04:12,858 : INFO : Finished: 40000 tags
2017-04-18 19:04:39,225 : INFO : Loading new file for index: 50000
2017-04-18 19:05:23,518 : INFO : Finished: 50000 tags
2017-04-18 19:05:48,791 : INFO : Loading new file for index: 60000
2017-04-18 19:06:35,336 : INFO : Finished: 60000 tags
2017-04-18 19:07:00,384 : INFO : Loading new file for index: 70000
2017-04-18 19:07:50,475 : INFO : Finished: 70000 tags
2017-04-18 19:08:15,

#### Testing inference

In [55]:
inference_docs_iterator = BatchWrapper(validation_preprocessed_files_prefix, batch_size=None, level=level, level_type=model_name)        
for doc_tuple in inference_doczs_iterator:
    doc_id, doc_tokens = doc_tuple
    rep = doc2vec_model.infer_vector(doc_tokens)
    print (doc_id, rep)
    break

('08521002', array([  1.13558674e+00,  -2.01971769e-01,  -9.30447519e-01,
         9.55632687e-01,   5.11517346e-01,   4.34441900e+00,
        -3.77764761e-01,  -1.11617422e+00,  -2.15896085e-01,
         9.09354746e-01,   5.74674904e-01,  -2.07049704e+00,
        -7.20400810e-01,   4.94136661e-01,  -1.74060893e+00,
        -2.17272949e+00,  -4.39270258e-01,  -1.51936769e+00,
         5.65607429e-01,  -4.58835810e-01,  -1.69598356e-01,
         1.77733886e+00,   3.66123140e-01,   1.38953611e-01,
        -1.04259264e+00,   8.84979665e-01,  -8.56729895e-02,
        -6.04329109e-01,   4.42179322e-01,   1.08561194e+00,
        -2.49654725e-01,   3.02951038e-01,  -3.80307257e-01,
         1.32433748e+00,   7.18038738e-01,   7.99864233e-01,
        -3.60305488e-01,  -3.32749695e-01,   1.86409019e-02,
        -1.16298962e+00,  -2.36521304e-01,   8.52507114e-01,
        -4.25269688e-03,  -2.73190904e+00,  -1.39228487e+00,
         3.14658254e-01,   1.19927609e+00,  -8.86219382e-01,
        -2.

In [53]:
doc2vec_model.wv.syn0

array([[ 0.24962339,  0.03808838, -0.38492572, ...,  0.81019139,
        -0.0872335 ,  0.00503489],
       [-0.24393913, -0.9072656 , -0.08245134, ..., -0.12438237,
        -0.10501056,  0.07241193],
       [ 0.06769085, -0.22004843,  0.05649997, ...,  0.15331532,
        -0.87121236, -0.71148068],
       ..., 
       [ 0.02257917,  0.18380728, -0.19475998, ...,  0.72972393,
        -0.03356596, -0.29145467],
       [-0.20255305, -0.25994578,  0.31640032, ..., -0.02623975,
         0.41660461, -0.45980361],
       [ 1.09419656, -0.97489876, -0.3509953 , ...,  0.82430571,
         0.02756385,  0.7905944 ]], dtype=float32)